# Prerequisites

In [ ]:
%tensorflow_version 1.x
!git clone https://github.com/imcaspar/gpt2-ml
%cd /content/gpt2-ml

# Download Pretrained Model

In [ ]:
import os
import io

from google.colab import auth
from googleapiclient.discovery import build
from apiclient.http import MediaIoBaseDownload
from tqdm import tqdm

auth.authenticate_user()
drive_service = build('drive', 'v3')

model_dir = '/content/gpt2-ml/models/mega'
if not os.path.exists(model_dir): os.makedirs(model_dir)

file_id, file_ext = ('1n_5-tgPpQ1gqbyLPbP1PwiFi2eo7SWw_', '.data-00000-of-00001')
filename = '%s/model.ckpt-%d%s' % (model_dir, 100000, file_ext)
req = drive_service.files().get_media(fileId=file_id)
with open(filename, 'wb') as f:
    downloader = MediaIoBaseDownload(f, req, chunksize=100*1024*1024)
    done = False
    pbar = tqdm(total=100, desc='%s' % filename)
    progress = 0
    while done is False:
        status, done = downloader.next_chunk()
        new_progress = int(status.progress() * 100)
        pbar.update(new_progress - progress)
        progress = new_progress
    pbar.close()

!wget -q --show-progress https://github.com/imcaspar/gpt2-ml/releases/download/v0.5/model.ckpt-100000.index -P models/mega
!wget -q --show-progress https://github.com/imcaspar/gpt2-ml/releases/download/v0.5/model.ckpt-100000.meta -P models/mega

# Inference

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
!PYTHONPATH=$(pwd) python3 scripts/interactive_conditional_samples.py -model_config_fn configs/mega.json -model_ckpt models/mega/model.ckpt-100000 -samples 10